In [ ]:
def replacemany(our_str, to_be_replaced:str, replace_with:str):
    while (to_be_replaced in our_str):
        our_str = our_str.replace(to_be_replaced, replace_with)
    return our_str

def _filter_dict(argsstr):
    if not argsstr or len(argsstr.strip())<1:
        return None
    env_dict={}
    argsstr=replacemany(replacemany(replacemany(argsstr.strip(),('  '),' '),('= '),'='),' =','=')
    envstr=str(str(argsstr.split(" ")).split("=")).replace(" ","").replace("\'","").replace("\"","").replace("[","").replace("]","").replace("\\","")
    env_list=envstr.split(",")
    for i in range(0,len(env_list),2):
        env_dict[str(env_list[i])]=env_list[i+1]
    return env_dict

envstr=' a1  =     1  b2=3     y3=6  '
print(replacemany(replacemany(replacemany(envstr.strip(),('  '),' '),('= '),'='),' =','='))
print(_filter_dict(envstr))


In [ ]:
from urllib.request import urlopen

myURL = urlopen("")
print(myURL.read())

In [8]:
## 
##%overwritefile
##%file:src/test.py
## %cleartest
print("------中文测试------")
##test_begin
print("------测试代码------\n")
##test_end

[MyPython] Info:file h:\Jupyter\ipynb\MyPython\src/test.py created successfully
[MyPython] Info:The process :h:\Jupyter\ipynb\MyPython\src/test.py
[MyPython] Info:python3 h:\Jupyter\ipynb\MyPython\src/test.py 


------中文测试------
------测试代码------



[MyPython] Info:The process end:14032


In [ ]:
import re
 
line = "//%Cat:s are smarter than dogs"
 
matchObj = re.match( r'(.*) are (.*?) .*', line)
matchObj1= re.search( r':(.*)',line)
if matchObj1:
    print ("matchObj1.group() : "+ matchObj1.group(0))
else:
   print ("No match!!")

value="cmd dd -g -o asdf"

args=[]
for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', value):
    args += [argument.strip('"')]
print(args.pop(0) +"\n")
print(args[0] +"\n")

#for argument in re.findall(r'(?:[^\s,"]|"(?:\\.|[^"])*")+', value):
    #print(argument)